# Stock Analysis Report Generator Using Perplexity

<small>

#### **Overview**
This notebook automatically generates professional stock analysis reports in the style of Brian Belsky (Chief Investment Strategist at BMO Capital Markets) using Perplexity AI's Sonar Pro model. Each report is saved as a formatted Microsoft Word document.6. The prompt used to generate the report
Analyst ratings and price targets (when available)

#### **Features**
- Batch Processing: Processes multiple stocks from a text file3. Key financial metrics table
- Smart Skipping: Avoids duplicate API calls by checking for existing reports2. 5-year price performance vs. NASDAQ
- Markdown Formatting: Converts AI-generated markdown to professional Word formatting1. Stock ticker and generation date
- Comprehensive Analysis: Includes price performance charts, financial metrics, and analyst ratingsEach report includes:
- Date Stamping: Automatically timestamps each report## Output Format

#### **Requirements**
- Perplexity API Key- 
- Input file: Text file with stock tickers (one per line)
- Python 3.7+

#### **Output**
- Analysis word document for each equity




## Import Libraries

In [1]:
# Check which packages are available and install missing ones
# Install from requirements file
%pip install -q -r requirements.txt
%pip install python-docx
print("✅ Package installation examples shown above!")

# Import necessary packages
import sys
import requests
from docx import Document
from docx.shared import Pt, RGBColor
from docx.enum.text import WD_PARAGRAPH_ALIGNMENT
import os
from dotenv import load_dotenv
from openai import OpenAI
import re
import os.path
from os.path import isfile, join
from os import listdir
import subprocess
import json
import sys
from datetime import datetime
from typing import Dict, List, Optional, Any
from IPython.display import Markdown, display
from IPython.display import SVG
import numpy as np
from time import time
np.random.seed(10)
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
import argparse
import random
import sys
from datetime import datetime, timedelta
import requests


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
✅ Package installation examples shown above!


## Configuration Settings

In [2]:
### Variable informational detail
# **MAX_TOKENS**: `2000` - Maximum length of generated response
# **INPUT_DIR**: Directory containing the equity list file- **TEMPERATURE**: `0` - Deterministic output for consistent, factual analysis
# **OUTPUT_DIR**: Directory where generated reports will be saved- **MODEL**: `sonar-pro` - Perplexity's most advanced model with real-time web access and current financial data
# **EQUITY_LIST_FILE**: Text file with stock tickers (one per line)
# **Model Settings: Specific model to be invoked, 0 temperature to eliminate creativity and limit the number of tokens for the query.

# Load environment variables
load_dotenv()

# Get API key
API_KEY = os.getenv("PERPLEXITY_API_KEY")

# Input/Output Configuration
INPUT_DIR = os.getenv("Input_dir")
OUTPUT_DIR_SEC_FILINGS = os.getenv("Output_dir_sec_filings")
OUTPUT_DIR_INDIVIDUAL_STOCK_ANALYSIS = os.getenv("Output_dir_individual_equities")
OUTPUT_DIR_PORTFOLIO_ANALYSIS = os.getenv("Output_dir_portfolio")
PROMPT_DIR = os.getenv("Prompt_dir")
EQUITY_LIST_FILE = os.getenv("EQUITY_LIST_FILE")
PROMPT_INDIVIDUAL_EQUITY_ANALYSIS = os.getenv("PROMPT_INDIVIDUAL_EQUITY_FILE")
PROMPT_PORTFOLIO_ANALYSIS = os.getenv("PROMPT_PORTFOLIO_FILE")
PROMPT_RATINGS_CHANGE=os.getenv("PROMPT_RATINGS_CHANGE_FILE")

# Read stock list from file
with open(EQUITY_LIST_FILE, 'r') as f:
    EQUITY_LIST = [line.strip() for line in f if line.strip()]
# print small selection of equities from list to verify 
#print(EQUITY_LIST)  # Print equities for verification
  
# Read prompt template from file
with open(PROMPT_INDIVIDUAL_EQUITY_ANALYSIS, 'r') as f:
    PROMPT_TEMPLATE = f.read().strip()
print(PROMPT_TEMPLATE)

# Read prompt template from file
with open(PROMPT_PORTFOLIO_ANALYSIS, 'r') as f:
    PROMPT_PORTFOLIO_TEMPLATE = f.read().strip()
print(" ")
print(PROMPT_PORTFOLIO_TEMPLATE)

# Read prompt template from file
with open(PROMPT_RATINGS_CHANGE, 'r') as f:
    PROMPT_RATINGS_CHANGE_TEMPLATE = f.read().strip()
print(" ")
print(PROMPT_RATINGS_CHANGE_TEMPLATE)

# Model Configuration
MODEL = "sonar-pro" 
TEMPERATURE = 0
MAX_TOKENS = 2000

# SEC code header
SEC_HEADER =os.getenv("User_Agent")  
print(SEC_HEADER)
 

give me a Brian Belsky style stock analysis for this actual stock market stock in a 3 page or less format. 
Insert a chart at the top with trailing 5 year stock price performance of this stock vs. the 
Nasdaq as well as a table of all the key financial metrics for this stock including the following key financial metrics

    Investment category/industry

    PE ratio (TTM)

    ROE (%)

    Dividend yield (%)

    Most recent revenue figure (USD billions)

    Beta

    Analyst recommendation
 Make certain all of the facts are correct.  

If available, please add analyst ratings and price targets for this stock or mutual fund.
 
"Taking all of these equities and other types of funds a comprehensive portfolio analysis covering the following equity and fund tickers:


Sort the table by industry category. For each ticker, include columns for:

    Investment category/industry

    PE ratio (TTM)

    ROE (%)

    Dividend yield (%)

    Most recent revenue figure (USD billions)

    Beta


## Define Class to Call Model API

In [3]:
from openai import OpenAI

class PerplexityClient:
    def __init__(self, api_key):
        self.client = OpenAI(
            api_key=api_key,
            base_url="https://api.perplexity.ai"
        )

    def chat(self, message, model=MODEL):
        response = self.client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}]
        )
        return response.choices[0].message.content

## Define Functions to Retrieve SEC Documents

In [4]:
# Fetch SEC CIK code for a given ticker symbol
def get_cik(ticker):
    """
    Retrieve the CIK (Central Index Key) for a given stock ticker from the SEC database.
    Returns the CIK as a zero-padded string if found, otherwise None.
    """
    url = "https://www.sec.gov/files/company_tickers_exchange.json"
    headers = {"User-Agent": SEC_HEADER}
    print("Fetching CIK for ticker:", ticker)
    
    resp = requests.get(url, headers=headers)
    if resp.status_code != 200:
        print(f"Failed to fetch CIK data. Status code: {resp.status_code}")
        return None
    try:
        data = resp.json()
    except Exception as e:
        print(f"Error decoding JSON: {e}")
        print("Response text:", resp.text[:200])
        return None
    # SEC JSON structure: {'fields': [...], 'data': [[...], ...]}
    if isinstance(data, dict) and "fields" in data and "data" in data:
        fields = data["fields"]
        data_list = data["data"]
        # Find the index for 'ticker' and for the CIK field (usually 'cik' or 'cik_str')
        ticker_idx = next((i for i, f in enumerate(fields) if f.lower() == "ticker"), None)
        cik_idx = next((i for i, f in enumerate(fields) if "cik" in f.lower()), None)
        if ticker_idx is None or cik_idx is None:
            print("Could not find required fields in SEC data.")
            return None
        for entry in data_list:
            if entry[ticker_idx].upper() == ticker.upper():
                return str(entry[cik_idx]).zfill(10)
        print(f"Ticker {ticker} not found in SEC database.")
        return None
    print("Unexpected SEC JSON structure.")
    return None

# Fetch recent SEC filings for a given CIK
def get_sec_filings(cik, forms=["10-K", "10-Q", "8-K"]):
    """
    Retrieve the most recent 10-K, 10-Q, and 8-K filings for a given CIK.
    Returns a list of up to 3 filings (one per form type).
    """
    base_url = f"https://data.sec.gov/submissions/CIK{str(cik).zfill(10)}.json"
    headers = {"User-Agent": SEC_HEADER}
    try:
        resp = requests.get(base_url, headers=headers)
        if resp.status_code != 200:
            print(f"Failed to fetch filings for CIK {cik}. Status code: {resp.status_code}")
            return []
        data = resp.json()
    except Exception as e:
        print(f"Error fetching filings for CIK {cik}: {e}")
        return []
    filings_dict = {}
    recent = data.get("filings", {}).get("recent", {})
    forms_list = recent.get("form", [])
    accession_list = recent.get("accessionNumber", [])
    filing_dates = recent.get("filingDate", [])
    primary_docs = recent.get("primaryDocument", [])
    for i, form in enumerate(forms_list):
        if form in forms and form not in filings_dict:
            filing = {
                "form": form,
                "date": filing_dates[i] if i < len(filing_dates) else None,
                "accession": accession_list[i] if i < len(accession_list) else None,
                "url": f"https://www.sec.gov/Archives/edgar/data/{int(cik)}/{accession_list[i].replace('-', '')}/{primary_docs[i]}" if i < len(accession_list) and i < len(primary_docs) else None,
            }
            filings_dict[form] = filing
            if len(filings_dict) == len(forms):
                break
    return [filings_dict[form] for form in forms if form in filings_dict]



def download_sec_filings(ticker, filings, output_dir):
    """
    Download SEC filings and save them to the specified output directory.
    Each file is named as: {ticker}_{form}_{date}.html
    """
    os.makedirs(output_dir, exist_ok=True)
    for filing in filings:
        url = filing.get("url")
        form = filing.get("form")
        date = filing.get("date")
        if not url or not form or not date:
            print(f"Skipping incomplete filing for {ticker}: {filing}")
            continue
        filename = f"{ticker}_{form}_{date}.html"
        filepath = os.path.join(output_dir, filename)
        try:
            resp = requests.get(url, headers={"User-Agent": SEC_HEADER})
            if resp.status_code == 200:
                with open(filepath, "wb") as f:
                    f.write(resp.content)
                print(f"Saved: {filepath}")
            else:
                print(f"Failed to download {url} (status {resp.status_code})")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

In [5]:
TICKER_LIST = EQUITY_LIST
print(EQUITY_LIST)
#OUTPUT_DIR_SEC_FILINGS
for ticker in TICKER_LIST:
    cik = get_cik(ticker)
    if not cik:
        continue
    filings = get_sec_filings(cik)
    for filing in filings:
        print(f"{filing['form']} | {filing['date']} | {filing['url']}")
    download_sec_filings(ticker, filings, OUTPUT_DIR_SEC_FILINGS)

['AMZN', 'RDDT', 'WBD', 'MSFT', 'VWICX', 'CEG', 'JPM', 'LLY', 'UBER', 'BBJP', 'VNM']
Fetching CIK for ticker: AMZN
10-K | 2025-02-07 | https://www.sec.gov/Archives/edgar/data/1018724/000101872425000004/amzn-20241231.htm
10-Q | 2025-10-31 | https://www.sec.gov/Archives/edgar/data/1018724/000101872425000123/amzn-20250930.htm
8-K | 2025-11-20 | https://www.sec.gov/Archives/edgar/data/1018724/000110465925114647/tm2530638d3_8k.htm
Saved: C:/Users/patty/portfolio_files/SEC_filings\AMZN_10-K_2025-02-07.html
Saved: C:/Users/patty/portfolio_files/SEC_filings\AMZN_10-Q_2025-10-31.html
Saved: C:/Users/patty/portfolio_files/SEC_filings\AMZN_8-K_2025-11-20.html
Fetching CIK for ticker: RDDT
10-K | 2025-02-13 | https://www.sec.gov/Archives/edgar/data/1713445/000171344525000018/rddt-20241231.htm
10-Q | 2025-10-31 | https://www.sec.gov/Archives/edgar/data/1713445/000171344525000227/rddt-20250930.htm
8-K | 2025-10-30 | https://www.sec.gov/Archives/edgar/data/1713445/000171344525000225/rddt-20251030.htm

## Define Classes to Format the Output in a Document

In [6]:
# Function to convert markdown to Word document content
def add_markdown_to_word(doc, markdown_text):
    """Convert markdown text to formatted Word document content"""
    lines = markdown_text.split('\n')
    i = 0
    
    while i < len(lines):
        line = lines[i]
        
        # Skip empty lines
        if not line.strip():
            i += 1
            continue
        
        # Headers (# ## ###)
        if line.startswith('#'):
            level = len(line) - len(line.lstrip('#'))
            text = line.lstrip('#').strip()
            doc.add_heading(text, level=min(level, 9))
        
        # Tables (|...)
        elif '|' in line and i + 1 < len(lines) and '|' in lines[i + 1]:
            # Parse table
            table_lines = [line]
            i += 1
            # Skip separator line
            if '---' in lines[i] or ':-:' in lines[i]:
                i += 1
            # Get table rows
            while i < len(lines) and '|' in lines[i]:
                table_lines.append(lines[i])
                i += 1
            
            # Create table
            headers = [cell.strip() for cell in table_lines[0].split('|') if cell.strip()]
            num_cols = len(headers)
            num_rows = len(table_lines)
            
            table = doc.add_table(rows=num_rows, cols=num_cols)
            table.style = 'Light Grid Accent 1'
            
            # Add headers
            for j, header in enumerate(headers):
                cell = table.rows[0].cells[j]
                cell.text = header
                cell.paragraphs[0].runs[0].bold = True
            
            # Add data rows
            for row_idx in range(1, len(table_lines)):
                cells = [cell.strip() for cell in table_lines[row_idx].split('|') if cell.strip()]
                for col_idx, cell_text in enumerate(cells):
                    if col_idx < num_cols:
                        table.rows[row_idx].cells[col_idx].text = cell_text
            
            doc.add_paragraph()  # Add space after table
            continue
        
        # Bullet points (- or *)
        elif line.strip().startswith(('- ', '* ', '• ')):
            text = line.strip()[2:].strip()
            # Handle bold/italic in bullet points
            p = doc.add_paragraph(style='List Bullet')
            add_formatted_text(p, text)
        
        # Numbered lists (1. 2. etc)
        elif re.match(r'^\d+\.\s', line.strip()):
            text = re.sub(r'^\d+\.\s', '', line.strip())
            p = doc.add_paragraph(style='List Number')
            add_formatted_text(p, text)
        
        # Bold text with **
        elif '**' in line or '__' in line:
            p = doc.add_paragraph()
            add_formatted_text(p, line)
        
        # Regular paragraph
        else:
            p = doc.add_paragraph()
            add_formatted_text(p, line)
        
        i += 1

def add_formatted_text(paragraph, text):
    """Add text with bold/italic markdown formatting to a paragraph"""
    # Handle **bold** and __bold__
    parts = re.split(r'(\*\*.*?\*\*|__.*?__|`.*?`)', text)
    
    for part in parts:
        if not part:
            continue
        
        if part.startswith('**') and part.endswith('**'):
            run = paragraph.add_run(part[2:-2])
            run.bold = True
        elif part.startswith('__') and part.endswith('__'):
            run = paragraph.add_run(part[2:-2])
            run.bold = True
        elif part.startswith('`') and part.endswith('`'):
            run = paragraph.add_run(part[1:-1])
            run.font.name = 'Courier New'
        else:
            paragraph.add_run(part)


def sort_markdown_table_by_industry(markdown_text):
    lines = markdown_text.split('\n')
    table_start = None
    table_end = None
    for i, line in enumerate(lines):
        if '|' in line and table_start is None:
            table_start = i
        elif table_start is not None and ('|' not in line or not line.strip()):
            table_end = i
            break
    if table_start is None or table_end is None:
        return markdown_text  # No table found

    header = lines[table_start]
    separator = lines[table_start + 1]
    rows = lines[table_start + 2:table_end]
    # Find the index of the Industry/Category column
    columns = [col.strip().lower() for col in header.split('|')]
    try:
        industry_idx = columns.index('industry')
    except ValueError:
        try:
            industry_idx = columns.index('category')
        except ValueError:
            return markdown_text  # No industry/category column

    # Sort rows by industry/category
    def get_industry(row):
        cells = [cell.strip() for cell in row.split('|')]
        return cells[industry_idx] if industry_idx < len(cells) else ''
    rows_sorted = sorted(rows, key=get_industry)

    # Rebuild the markdown
    sorted_table = [header, separator] + rows_sorted
    lines = lines[:table_start] + sorted_table + lines[table_end:]
    return '\n'.join(lines)

## Generate Analysis for Each Stock in a List

In [7]:
# Ensure output directory exists
os.makedirs(OUTPUT_DIR_INDIVIDUAL_STOCK_ANALYSIS, exist_ok=True)  

# Initialize Perplexity client
client = PerplexityClient(api_key=API_KEY)
counter = 0
# Process each equity
for equity in EQUITY_LIST:
 
    # Construct prompt with equity ticker and template from file
    prompt = f"For the equity {equity} {PROMPT_TEMPLATE}"
    
    # Check if report already exists for today
    date_str = datetime.now().strftime('%Y-%m-%d')
    output_filename = f"Equity Report - {equity} {date_str}.docx"
    output_path = os.path.join(OUTPUT_DIR_INDIVIDUAL_STOCK_ANALYSIS, output_filename)
 
    if os.path.exists(output_path):
        #print(f"⏭️  Skipping {equity} - report already exists for {date_str}")
        continue

    try:
        # Query the LLM
        print(f"Querying Perplexity API for equities for {equity} ({counter + 1}/{len(EQUITY_LIST)})...")
        # print("Using prompt:",prompt   )

        generated_text = client.chat(prompt, model=MODEL)
        counter += 1
        
        # Create Word document
        doc = Document()
        doc.add_heading(f"Market Outlook Report - {equity}", 0)
        doc.add_paragraph(f"Perplexity Sonar Pro Model Generated: {date_str}")
        doc.add_paragraph()
        
        # Convert markdown to Word formatting
        add_markdown_to_word(doc, generated_text)
        
        # Add prompt at the end
        doc.add_paragraph()
        doc.add_heading("Prompt Used:", level=2)
        doc.add_paragraph(prompt)
        
        # Save the document
        doc.save(output_path)
        # print(f"✅ Saved: {output_filename}")
        
    except Exception as e:
        print(f"❌ Error processing {equity}: {e}")
        continue

print(f"\n{'='*60}")
print(f"✅ Completed processing {len(EQUITY_LIST)} equities")
print(f"Reports saved to: {OUTPUT_DIR_INDIVIDUAL_STOCK_ANALYSIS}")
print(f"{'='*60}")


Querying Perplexity API for equities for AMZN (1/11)...
Querying Perplexity API for equities for RDDT (2/11)...
Querying Perplexity API for equities for WBD (3/11)...
Querying Perplexity API for equities for MSFT (4/11)...
Querying Perplexity API for equities for VWICX (5/11)...
Querying Perplexity API for equities for CEG (6/11)...
Querying Perplexity API for equities for JPM (7/11)...
Querying Perplexity API for equities for LLY (8/11)...
Querying Perplexity API for equities for UBER (9/11)...
Querying Perplexity API for equities for BBJP (10/11)...
Querying Perplexity API for equities for VNM (11/11)...

✅ Completed processing 11 equities
Reports saved to: C:/Users/patty/portfolio_files/Individual_stock_analysis


## Generate Portfolio Analysis

In [8]:
from datetime import datetime
import os
from docx.shared import Pt

# Ensure output directory exists
os.makedirs(OUTPUT_DIR_PORTFOLIO_ANALYSIS, exist_ok=True)

# Initialize Perplexity client with API key from environment variable
client = PerplexityClient(api_key=API_KEY)

# Generate a single prompt for the entire EQUITY_LIST using PROMPT_PORTFOLIO_ANALYSIS
date_str = datetime.now().strftime('%Y-%m-%d')
output_filename = f"Portfolio Report {date_str}.docx"
output_path = os.path.join(OUTPUT_DIR_PORTFOLIO_ANALYSIS, output_filename)

# Combine all equities into a single string (comma separated, or as a list)
equity_str = ', '.join(EQUITY_LIST)

# Read the portfolio prompt template from file (already done above)
# with open(PROMPT_PORTFOLIO_ANALYSIS, 'r') as f:
#     PROMPT_PORTFOLIO_TEMPLATE = f.read().strip()

# Construct the prompt for the entire list
prompt = f"For the equities {equity_str} {PROMPT_PORTFOLIO_TEMPLATE}"
print(f"Processing portfolio for select equities and generating individual reports")

# Check if report already exists for today
if os.path.exists(output_path):
    print(f"⏭️  Skipping portfolio - report already exists for {date_str}")
else:
    try:
        # Query the LLM
        # print(f"Querying Perplexity API for portfolio...")
        generated_text = client.chat(prompt, model=MODEL)

        # Sort the markdown table by Industry/Category
        sorted_markdown = sort_markdown_table_by_industry(generated_text)

        # Create Word document
        doc = Document()
        doc.add_heading(f"Market Outlook Portfolio Report", 0)
        doc.add_paragraph(f"Perplexity Sonar Pro Model Generated: {date_str}")
        doc.add_paragraph()

        # Convert markdown to Word formatting
        add_markdown_to_word(doc, sorted_markdown)

        # Apply formatting to the last table (smaller font, no bold, remove asterisks)
        if doc.tables:
            table = doc.tables[-1]
            for row in table.rows:
                for cell in row.cells:
                    for paragraph in cell.paragraphs:
                        for run in paragraph.runs:
                            run.font.size = Pt(8)
                            run.bold = False
                            run.text = run.text.replace('*', '')

        # Add prompt at the end
        doc.add_paragraph()
        doc.add_heading("Prompt Used:", level=2)
        doc.add_paragraph(prompt)

        # Save the document
        doc.save(output_path)
        print(f"✅ Saved: {output_filename}")

    except Exception as e:
        print(f"❌ Error processing portfolio: {e}")


Processing portfolio for select equities and generating individual reports
⏭️  Skipping portfolio - report already exists for 2025-12-05


## Utility to Format Table

In [9]:
# --- Utility to sort markdown table by a given column and remove bolds/asterisks ---
def sort_and_clean_markdown_table(markdown_text):
    sort_column_name = 'Current Consensus Analyst Rating'
    lines = markdown_text.split('\n')
    table_start = None
    table_end = None
    for i, line in enumerate(lines):
        if '|' in line and table_start is None:
            table_start = i
        elif table_start is not None and ('|' not in line or not line.strip()):
            table_end = i
            break
    if table_start is None or table_end is None:
        return markdown_text  # No table found

    header = lines[table_start]
    separator = lines[table_start + 1]
    rows = lines[table_start + 2:table_end]

    # Remove bolds and asterisks from header and rows
    def clean(text):
        return text.replace('**', '').replace('*', '').replace('__', '')
    header_clean = clean(header)
    rows_clean = [clean(row) for row in rows]
    columns = [col.strip().lower() for col in header_clean.split('|')]
    try:
        sort_idx = columns.index(sort_column_name.lower())
    except ValueError:
        return '\n'.join([header_clean, separator] + rows_clean + lines[table_end:])
    def get_sort_key(row):
        cells = [cell.strip() for cell in row.split('|')]
        return cells[sort_idx] if sort_idx < len(cells) else ''
    rows_sorted = sorted(rows_clean, key=get_sort_key)
    sorted_table = [header_clean, separator] + rows_sorted
    lines = lines[:table_start] + sorted_table + lines[table_end:]
    return '\n'.join(lines)

## Generate Ratings Change Report and Notifications

In [10]:
# Generate Ratings Change Report for select equities
from docx import Document
from datetime import datetime
import os
import time

# Ensure output directory exists
os.makedirs(OUTPUT_DIR_PORTFOLIO_ANALYSIS, exist_ok=True)

date_str = datetime.now().strftime('%Y-%m-%d')
output_filename = f"Ratings Change Report {date_str}.docx"
output_path = os.path.join(OUTPUT_DIR_PORTFOLIO_ANALYSIS, output_filename)

doc = Document()
doc.add_heading("Ratings Change Report", 0)
doc.add_paragraph(f"Perplexity Sonar Pro Model Generated: {date_str}")
doc.add_paragraph()

#Summary table API call time: {t1-t0:.2f} seconds
# --- Individual equity details ---

for equity in EQUITY_LIST:
    prompt = f"For the equity {equity} {PROMPT_RATINGS_CHANGE_TEMPLATE}"
    doc.add_heading(f"{equity}", level=1)
    try:
        t0 = time.time()
        generated_text = client.chat(prompt, model=MODEL)
        t1 = time.time()
        add_markdown_to_word(doc, generated_text)
        # print(f"{equity} API call time: {t1-t0:.2f} seconds")
    except Exception as e:
        doc.add_paragraph(f"❌ Error processing {equity}: {e}")
    doc.add_paragraph()  # Space between equities

# Add prompt template at the end for reference
doc.add_heading("Prompt Template Used:", level=2)
doc.add_paragraph(PROMPT_RATINGS_CHANGE_TEMPLATE)

doc.save(output_path)
print(f"✅ Saved: {output_filename} in {OUTPUT_DIR_PORTFOLIO_ANALYSIS}")

✅ Saved: Ratings Change Report 2025-12-05.docx in C:/Users/patty/portfolio_files/Portfolio_analysis


## End